In [1]:
import os
import time
import datetime
import csv
import json
from math import sqrt
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 参数配置

class TrainingConfig(object):
    epoches = 20
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.0005
    

class ModelConfig(object):
    
    # 该列表中子列表的三个元素分别是卷积核的数量，卷积核的高度，池化的尺寸
    convLayers = [[256, 7, 4],
                  [256, 7, 4],
                  [256, 3, 4]]
#                   [256, 3, None],
#                   [256, 3, None],
#                   [256, 3, 3]]
    fcLayers = [512]
    dropoutKeepProb = 0.5
    
    
class Config(object):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
#     alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    
    sequenceLength = 1014
    batchSize = 128
    
    rate = 0.8  # 训练集的比例
    
    dataSource = "../data/preProcess/labeledCharTrain.csv"
    
    training = TrainingConfig()
    
    model = ModelConfig()

config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        
        self._sequenceLength = config.sequenceLength
        self._rate = config.rate
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self._alphabet = config.alphabet
        self.charEmbedding =None
        
        self._charToIndex = {}
        self._indexToChar = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [[char for char in line if char != " "] for line in review]
        
        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, charToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in charToIndex:
                reviewVec[i] = charToIndex[review[i]]
            else:
                reviewVec[i] = charToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._charToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成字符向量和字符-索引映射字典
        """
        
        chars = [char for char in self._alphabet]
        
        vocab, charEmbedding = self._getCharEmbedding(chars)
        self.charEmbedding = charEmbedding
        
        self._charToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToChar = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/charJson/charToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._charToIndex, f)
        
        with open("../data/charJson/indexToChar.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToChar, f)
            
    def _getCharEmbedding(self, chars):
        """
        按照one的形式将字符映射成向量
        """
        
        alphabet = ["UNK"] + [char for char in self._alphabet]
        vocab = ["pad"] + alphabet
        charEmbedding = []
        charEmbedding.append(np.zeros(len(alphabet), dtype="float32"))
        
        for i, alpha in enumerate(alphabet):
            onehot = np.zeros(len(alphabet), dtype="float32")
            
            # 生成每个字符对应的向量
            onehot[i] = 1
            
            # 生成字符嵌入的向量矩阵
            charEmbedding.append(onehot)
                
        return vocab, np.array(charEmbedding)
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))
print("charEmbedding shape: {}".format(data.charEmbedding.shape))

train data shape: (20000, 1014)
train label shape: (20000, 1)
eval data shape: (5000, 1014)
charEmbedding shape: (71, 70)


In [6]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = (len(x) - 1) // batchSize

        for i in range(numBatches - 1):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [12]:
# 定义char-CNN分类器

class CharCNN(object):
    """
    char-CNN用于文本分类
    """
    def __init__(self, config, charEmbedding):
        # placeholders for input, output and dropuot
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.is_training = tf.placeholder(tf.bool, name="is_training")
        
        # 字符嵌入
        with tf.name_scope("embedding"):
            
            # 利用one-hot的字符向量作为初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(charEmbedding, dtype=tf.float32, name="charEmbedding") ,name="W")
            # 获得字符嵌入
            self.embededChars = tf.nn.embedding_lookup(self.W, self.inputX)
            # 添加一个通道维度
            self.embededCharsExpand = tf.expand_dims(self.embededChars, -1)

        for i, cl in enumerate(config.model.convLayers):
            print("开始第" + str(i + 1) + "卷积层的处理")
            # 利用命名空间name_scope来实现变量名复用
            with tf.name_scope("convLayer-%s"%(i+1)):
                # 获取字符的向量长度
                filterWidth = self.embededCharsExpand.get_shape()[2].value
                
                # filterShape = [height, width, in_channels, out_channels]
                filterShape = [cl[1], filterWidth, 1, cl[0]]

                stdv = 1 / sqrt(cl[0] * cl[1])
                
                # 初始化w和b的值
                wConv = tf.Variable(tf.random_uniform(filterShape, minval=-stdv, maxval=stdv),
                                     dtype='float32', name='w')
                bConv = tf.Variable(tf.random_uniform(shape=[cl[0]], minval=-stdv, maxval=stdv), name='b')
                
#                 w_conv = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.05), name="w")
#                 b_conv = tf.Variable(tf.constant(0.1, shape=[cl[0]]), name="b")
                # 构建卷积层，可以直接将卷积核的初始化方法传入（w_conv）
                conv = tf.nn.conv2d(self.embededCharsExpand, wConv, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                # 加上偏差
                hConv = tf.nn.bias_add(conv, bConv)
                # 可以直接加上relu函数，因为tf.nn.conv2d事实上是做了一个卷积运算，然后在这个运算结果上加上偏差，再导入到relu函数中
                
                # h_conv = tf.nn.relu(h_conv)
                if cl[-1] is not None:
                    ksizeShape = [1, cl[2], 1, 1]
                    hPool = tf.nn.max_pool(hConv, ksize=ksizeShape, strides=ksizeShape, padding="VALID", name="pool")
                else:
                    hPool = hConv
                    
                print(hPool.shape)
    
                # 对维度进行转换，转换成卷积层的输入维度
                self.embededCharsExpand = tf.transpose(hPool, [0, 1, 3, 2], name="transpose")
        print(self.embededCharsExpand)
        with tf.name_scope("reshape"):
            fcDim = self.embededCharsExpand.get_shape()[1].value * self.embededCharsExpand.get_shape()[2].value
            self.inputReshape = tf.reshape(self.embededCharsExpand, [-1, fcDim])
        
        # 保存的是神经元的个数[34*256, 1024, 1024]
        weights = [fcDim] + config.model.fcLayers
        
        for i, fl in enumerate(config.model.fcLayers):
            with tf.name_scope("fcLayer-%s"%(i+1)):
                print("开始第" + str(i + 1) + "全连接层的处理")
                stdv = 1 / sqrt(weights[i])
                
                # 定义全连接层的初始化方法，均匀分布初始化w和b的值
                wFc = tf.Variable(tf.random_uniform([weights[i], fl], minval=-stdv, maxval=stdv), dtype="float32", name="w")
                bFc = tf.Variable(tf.random_uniform(shape=[fl], minval=-stdv, maxval=stdv), dtype="float32", name="b")
                
#                 w_fc = tf.Variable(tf.truncated_normal([weights[i], fl], stddev=0.05), name="W")
#                 b_fc = tf.Variable(tf.constant(0.1, shape=[fl]), name="b")
                
                self.fcInput = tf.nn.relu(tf.matmul(self.inputReshape, wFc) + bFc)
                
                with tf.name_scope("dropOut"):
                    self.fcInputDrop = tf.nn.dropout(self.fcInput, self.dropoutKeepProb)
                    
            self.inputReshape = self.fcInputDrop
            
        with tf.name_scope("outputLayer"):
            stdv = 1 / sqrt(weights[-1])
            # 定义隐层到输出层的权重系数和偏差的初始化方法
#             w_out = tf.Variable(tf.truncated_normal([fc_layers[-1], num_classes], stddev=0.1), name="W")
#             b_out = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            
            wOut = tf.Variable(tf.random_uniform([config.model.fcLayers[-1], 1], minval=-stdv, maxval=stdv), dtype="float32", name="w")
            bOut = tf.Variable(tf.random_uniform(shape=[1], minval=-stdv, maxval=stdv), name="b")
            # tf.nn.xw_plus_b就是x和w的乘积加上b
            self.predictions = tf.nn.xw_plus_b(self.inputReshape, wOut, bOut, name="predictions")
            # 进行二元分类
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
            
            
        with tf.name_scope("loss"):
            # 定义损失函数，对预测值进行softmax，再求交叉熵。
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses)
    
    def batch_norm(self, x):
        
        gamma = tf.Variable(tf.ones([x.get_shape()[3].value]))
        beta = tf.Variable(tf.zeros([x.get_shape()[3].value]))

        pop_mean = tf.Variable(tf.zeros([x.get_shape()[3].value]), trainable=False)
        pop_variance = tf.Variable(tf.ones([x.get_shape()[3].value]), trainable=False)

        epsilon = 1e-3

        def batch_norm_training():
            # 一定要使用正确的维度确保计算的是每个特征图上的平均值和方差而不是整个网络节点上的统计分布值
            batch_mean, batch_variance = tf.nn.moments(layer, [0, 1, 2], keep_dims=False)

            decay = 0.99
            train_mean = tf.assign(pop_mean, pop_mean*decay + batch_mean*(1 - decay))
            train_variance = tf.assign(pop_variance, pop_variance*decay + batch_variance*(1 - decay))

            with tf.control_dependencies([train_mean, train_variance]):
                return tf.nn.batch_normalization(x, batch_mean, batch_variance, beta, gamma, epsilon)

        def batch_norm_inference():
            return tf.nn.batch_normalization(x, pop_mean, pop_variance, beta, gamma, epsilon)

        batch_normalized_output = tf.cond(self.is_training, batch_norm_training, batch_norm_inference)
        return tf.nn.relu(batch_normalized_output)

In [8]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY, average='macro')
    recall = recall_score(trueY, binaryPredY, average='macro')
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [13]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

charEmbedding = data.charEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        
        cnn = CharCNN(config, charEmbedding)
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.RMSPropOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(cnn.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("trainLoss", cnn.loss)
        
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/charCNN/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/charCNN/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(cnn.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(cnn.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(cnn.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

开始第1卷积层的处理
(?, 252, 1, 256)
开始第2卷积层的处理
(?, 61, 1, 256)
开始第3卷积层的处理
(?, 14, 1, 256)
Tensor("convLayer-3/transpose:0", shape=(?, 14, 256, 1), dtype=float32)
开始第1全连接层的处理
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-1/w:0/grad/hist is illegal; using convLayer-1/w_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-1/w:0/grad/sparsity is illegal; using convLayer-1/w_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-1/b:0/grad/hist is illegal; using convLayer-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-1/b:0/grad/sparsity is illegal; using convLayer-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-2/w:0/grad/hist is illegal; using convLayer-2/w_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-2/w:0/grad/sparsity is 

2018-12-27T14:15:09.016315, step: 47, loss: 0.694043755531311, acc: 0.4922, auc: 0.3963, precision: 0.2461, recall: 0.5
2018-12-27T14:15:09.133129, step: 48, loss: 0.6945300102233887, acc: 0.4531, auc: 0.4052, precision: 0.2266, recall: 0.5
2018-12-27T14:15:09.250021, step: 49, loss: 0.6936608552932739, acc: 0.5234, auc: 0.3861, precision: 0.2617, recall: 0.5
2018-12-27T14:15:09.366834, step: 50, loss: 0.6926628351211548, acc: 0.5547, auc: 0.447, precision: 0.2773, recall: 0.5
2018-12-27T14:15:09.481715, step: 51, loss: 0.6923799514770508, acc: 0.5469, auc: 0.5074, precision: 0.2734, recall: 0.5
2018-12-27T14:15:09.600520, step: 52, loss: 0.6922101974487305, acc: 0.5625, auc: 0.4774, precision: 0.2812, recall: 0.5
2018-12-27T14:15:09.720100, step: 53, loss: 0.692521333694458, acc: 0.5625, auc: 0.4095, precision: 0.2812, recall: 0.5
2018-12-27T14:15:09.852721, step: 54, loss: 0.6924678087234497, acc: 0.5156, auc: 0.5455, precision: 0.2578, recall: 0.5
2018-12-27T14:15:09.976903, step: 5

2018-12-27T14:15:21.842364, step: 114, loss: 0.6930699944496155, acc: 0.4922, auc: 0.5031, precision: 0.2461, recall: 0.5
2018-12-27T14:15:21.955183, step: 115, loss: 0.6941208839416504, acc: 0.4297, auc: 0.4127, precision: 0.2148, recall: 0.5
2018-12-27T14:15:22.076716, step: 116, loss: 0.694159984588623, acc: 0.5312, auc: 0.3735, precision: 0.2656, recall: 0.5
2018-12-27T14:15:22.207930, step: 117, loss: 0.6935352087020874, acc: 0.4688, auc: 0.4662, precision: 0.2344, recall: 0.5
2018-12-27T14:15:22.346200, step: 118, loss: 0.6937286257743835, acc: 0.4531, auc: 0.3936, precision: 0.2266, recall: 0.5
2018-12-27T14:15:22.476446, step: 119, loss: 0.6928616166114807, acc: 0.4453, auc: 0.4539, precision: 0.2227, recall: 0.5
2018-12-27T14:15:22.592044, step: 120, loss: 0.6934188604354858, acc: 0.4844, auc: 0.4174, precision: 0.2422, recall: 0.5
2018-12-27T14:15:22.709868, step: 121, loss: 0.6936219930648804, acc: 0.5156, auc: 0.4844, precision: 0.2578, recall: 0.5
2018-12-27T14:15:22.84698

2018-12-27T14:15:29.652119, step: 182, loss: 0.6909480094909668, acc: 0.4766, auc: 0.5738, precision: 0.2383, recall: 0.5
2018-12-27T14:15:29.753282, step: 183, loss: 0.6898432970046997, acc: 0.4688, auc: 0.5478, precision: 0.2344, recall: 0.5
2018-12-27T14:15:29.847933, step: 184, loss: 0.6969574093818665, acc: 0.5234, auc: 0.5481, precision: 0.2617, recall: 0.5
2018-12-27T14:15:29.952252, step: 185, loss: 0.6952588558197021, acc: 0.5078, auc: 0.402, precision: 0.2539, recall: 0.5
2018-12-27T14:15:30.063121, step: 186, loss: 0.6924149990081787, acc: 0.4766, auc: 0.541, precision: 0.2383, recall: 0.5
2018-12-27T14:15:30.171099, step: 187, loss: 0.6926394701004028, acc: 0.5234, auc: 0.5356, precision: 0.2617, recall: 0.5
2018-12-27T14:15:30.266523, step: 188, loss: 0.6923314332962036, acc: 0.5391, auc: 0.5107, precision: 0.2695, recall: 0.5
2018-12-27T14:15:30.370595, step: 189, loss: 0.6978334188461304, acc: 0.4453, auc: 0.4865, precision: 0.2227, recall: 0.5
2018-12-27T14:15:30.467894

2018-12-27T14:15:40.941624, step: 249, loss: 0.6915631294250488, acc: 0.4844, auc: 0.5767, precision: 0.2422, recall: 0.5
2018-12-27T14:15:41.053937, step: 250, loss: 0.6907268762588501, acc: 0.4844, auc: 0.5496, precision: 0.2422, recall: 0.5
2018-12-27T14:15:41.165045, step: 251, loss: 0.6918008327484131, acc: 0.4766, auc: 0.5168, precision: 0.2383, recall: 0.5
2018-12-27T14:15:41.284059, step: 252, loss: 0.6911479830741882, acc: 0.5625, auc: 0.6101, precision: 0.2812, recall: 0.5
2018-12-27T14:15:41.419983, step: 253, loss: 0.6882790327072144, acc: 0.5625, auc: 0.5129, precision: 0.2812, recall: 0.5
2018-12-27T14:15:41.535734, step: 254, loss: 0.6875301599502563, acc: 0.5234, auc: 0.6491, precision: 0.2617, recall: 0.5
2018-12-27T14:15:41.654952, step: 255, loss: 0.6947717666625977, acc: 0.5078, auc: 0.4987, precision: 0.2539, recall: 0.5
2018-12-27T14:15:41.775182, step: 256, loss: 0.6922111511230469, acc: 0.4688, auc: 0.5576, precision: 0.2344, recall: 0.5
2018-12-27T14:15:41.8991

...

2018-12-27T14:23:33.801913, step: 2824, loss: 0.14911283552646637, acc: 0.9609, auc: 1.0, precision: 0.9597, recall: 0.9648
2018-12-27T14:23:33.947969, step: 2825, loss: 0.13171522319316864, acc: 0.9453, auc: 1.0, precision: 0.9573, recall: 0.934
2018-12-27T14:23:34.089272, step: 2826, loss: 0.024675441905856133, acc: 0.9844, auc: 1.0, precision: 0.9867, recall: 0.9818
2018-12-27T14:23:34.222463, step: 2827, loss: 0.015081522986292839, acc: 0.9922, auc: 1.0, precision: 0.9922, recall: 0.9923
2018-12-27T14:23:34.350808, step: 2828, loss: 0.0046531944535672665, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:34.479999, step: 2829, loss: 0.0006835561944171786, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:34.609117, step: 2830, loss: 0.026470297947525978, acc: 0.9922, auc: 0.9995, precision: 0.9923, recall: 0.9922
2018-12-27T14:23:34.747557, step: 2831, loss: 0.03218584135174751, acc: 0.9844, auc: 0.9995, precision: 0.9843, recall: 0.9843
2018-12-27T14:

2018-12-27T14:23:43.031558, step: 2892, loss: 0.0865529403090477, acc: 0.9688, auc: 0.998, precision: 0.9701, recall: 0.9673
2018-12-27T14:23:43.155953, step: 2893, loss: 0.2167140245437622, acc: 0.9297, auc: 0.9988, precision: 0.9338, recall: 0.9348
2018-12-27T14:23:43.275409, step: 2894, loss: 0.12203770875930786, acc: 0.9688, auc: 0.9985, precision: 0.9737, recall: 0.9643
2018-12-27T14:23:43.396341, step: 2895, loss: 0.06173110753297806, acc: 0.9531, auc: 0.9985, precision: 0.9541, recall: 0.9541
2018-12-27T14:23:43.514176, step: 2896, loss: 0.014713570475578308, acc: 0.9922, auc: 1.0, precision: 0.9924, recall: 0.9921
2018-12-27T14:23:43.645181, step: 2897, loss: 0.0081546725705266, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:43.773655, step: 2898, loss: 0.012718499638140202, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:43.902936, step: 2899, loss: 0.009471697732806206, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:44.0391

2018-12-27T14:23:56.762381, step: 2958, loss: 0.016681790351867676, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:56.891675, step: 2959, loss: 0.06442627310752869, acc: 0.9844, auc: 1.0, precision: 0.9848, recall: 0.9844
2018-12-27T14:23:57.020900, step: 2960, loss: 0.013350773602724075, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.162471, step: 2961, loss: 0.010183863341808319, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.294277, step: 2962, loss: 0.00548678170889616, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.433262, step: 2963, loss: 0.00365798594430089, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.576379, step: 2964, loss: 0.0025451891124248505, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.713808, step: 2965, loss: 0.004744087345898151, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:23:57.858726, step: 2966, loss: 0.01539683248847723

2018-12-27T14:24:10.616267, step: 3024, loss: 0.007029212545603514, acc: 0.9922, auc: 1.0, precision: 0.9924, recall: 0.9921
2018-12-27T14:24:10.744036, step: 3025, loss: 0.0319538339972496, acc: 0.9844, auc: 0.9995, precision: 0.9843, recall: 0.9843
2018-12-27T14:24:10.883331, step: 3026, loss: 0.04600795358419418, acc: 0.9844, auc: 1.0, precision: 0.9841, recall: 0.9851
2018-12-27T14:24:11.012625, step: 3027, loss: 0.03411990404129028, acc: 0.9844, auc: 0.9997, precision: 0.9839, recall: 0.9839
2018-12-27T14:24:11.158625, step: 3028, loss: 0.006250094622373581, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:24:11.288171, step: 3029, loss: 0.03493577986955643, acc: 0.9922, auc: 0.9993, precision: 0.9922, recall: 0.9923
2018-12-27T14:24:11.428027, step: 3030, loss: 0.04926406592130661, acc: 0.9844, auc: 0.999, precision: 0.9843, recall: 0.9843
2018-12-27T14:24:11.554412, step: 3031, loss: 0.0027616608422249556, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T1

2018-12-27T14:24:19.866526, step: 3092, loss: 0.0424620546400547, acc: 0.9844, auc: 0.9995, precision: 0.9843, recall: 0.9843
2018-12-27T14:24:19.995123, step: 3093, loss: 0.01494593732059002, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:24:20.124805, step: 3094, loss: 0.02443850226700306, acc: 0.9844, auc: 0.9998, precision: 0.9844, recall: 0.9844
2018-12-27T14:24:20.261450, step: 3095, loss: 0.007260921411216259, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:24:20.403155, step: 3096, loss: 0.005617574788630009, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:24:20.539291, step: 3097, loss: 0.02310865931212902, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2018-12-27T14:24:20.677944, step: 3098, loss: 0.08150719106197357, acc: 0.9688, auc: 0.9985, precision: 0.9733, recall: 0.9649
2018-12-27T14:24:20.822322, step: 3099, loss: 0.06479298323392868, acc: 0.9844, auc: 0.9988, precision: 0.9861, recall: 0.9828
2018-12-27T14:24:20.967336, ste